In [1]:
USE_SAME_FEW_SHOTS=True

In [2]:
number_of_shots = 5

In [3]:
# model_name = "meta-llama/Llama-2-7b-chat-hf"
model_name = "meta-llama/Llama-2-13b-chat-hf"

In [4]:
!pip install -q openai --progress-bar off

In [5]:
!pip install -q langchain --progress-bar off

In [6]:
import pandas as pd

import datetime
from langchain import PromptTemplate
from langchain import LLMChain
from langchain.chat_models import ChatOpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.prompts import PromptTemplate

import os
import json
import ast
import re

import pickle

import glob

# import openai
# from openai import OpenAI

from typing import Union
from IPython.display import display, HTML

from sklearn.metrics import classification_report

In [7]:
import os
from IPython.display import clear_output

os.environ["OPENAI_API_BASE"] = 'https://api.endpoints.anyscale.com/v1'
os.environ["OPENAI_API_KEY"] = input("INSERT YOUR ANYSCALE API KEY:")

clear_output()

In [8]:
output_datapath = "data/output/2023.11.28-5_Llama2_13B_API_5-few-shot_SEMANTIC_SIMILARITY/"

In [9]:
# # llm = openai.ChatCompletion.create(
# #     model = 'meta-llama/Llama-2-7b-chat-hf', 
# #     messages = [{"role": "system", "content": prompt}],
# #     max_tokens = 150,
# #     response_format = {"type": "json_object"},
# #     stream = True
# # )

# client = OpenAI()

# chat_completion = client.chat.completions.create(
#     messages=[
#         {
#             "role": "user",
#             "content": "Say this is a test",
#         }
#     ],
#     model='meta-llama/Llama-2-7b-chat-hf',
# )

In [10]:
import os
import requests

s = requests.Session()

api_base = os.getenv("OPENAI_API_BASE")
token = os.getenv("OPENAI_API_KEY")
url = f"{api_base}/chat/completions"

In [11]:
# if USE_SAME_FEW_SHOTS==True:
#     data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN.xlsx").drop(columns=['Unnamed: 0'])
#     data_sel_4_few_shot_examples = data_sel_4_few_shot_examples[
#         (data_sel_4_few_shot_examples.text_id==5614) |
#         (data_sel_4_few_shot_examples.text_id==2089) |
#         (data_sel_4_few_shot_examples.text_id==5665) |
#         (data_sel_4_few_shot_examples.text_id==370)  |
#         (data_sel_4_few_shot_examples.text_id==1422) |
#         (data_sel_4_few_shot_examples.text_id==4329) |
#         (data_sel_4_few_shot_examples.text_id==4929)
#     ]
# else:
#     data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN.xlsx").drop(columns=['Unnamed: 0']).sample(5)

# data_sel_4_few_shot_examples

In [12]:
top7_train_data_sim_to_each_test_data_filename = "../data/input/top7_train_data_sim_to_each_test_data.pickle"

with open(top7_train_data_sim_to_each_test_data_filename, 'rb') as f:
    top7_train_data_sim_to_each_test_data = pickle.load(f)

In [13]:
# few_shots = ""

# for _, row in data_sel_4_few_shot_examples[['text', 'sdg']].iterrows():
#     input_text = "Input text: \""+row['text']+"\"\n"
#     output_text = "Output: \"---SDG "+str(row['sdg'])+'---\"\n\n'
    
#     few_shots += input_text
#     few_shots += output_text
    
# print(few_shots)

In [14]:
data = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_1020_texts_TEST.xlsx")
data = data.drop(columns=['Unnamed: 0']) #, 'goal_label', 'motivation_for_the_answer', 'goal_label_int', 'sdg_probability'])
data

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN
...,...,...,...,...,...,...,...
1015,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0
1016,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0
1017,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0
1018,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0


In [15]:
data_sel_4_few_shot_examples = pd.read_excel("../data/input/sdg_17_labels_classification_dataset_4760_texts_TRAIN.xlsx")

In [16]:
# template_part1 = """[INST]
# <<SYS>>
# In the following lines, a text enclosed in triple quotes will be given to you.

# The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

# The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

# If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "---SDG 0---".

# The following are 7 examples of expected classification given an input text:

# """+few_shots+"""

# The text to be processed is:
# <</SYS>>

# '''
# """

# template_part2 = """
# '''

# <<SYS>>
# Answer with label only usign the format:---LABEL---.
# <</SYS>>
# [/INST]
# """

In [17]:
#data.iloc[181:183]

In [18]:
# data_sel = data #.sample(2)
data_sel = data #.iloc[54:] #.sample(2)

In [19]:
# body = {
#   "model": "meta-llama/Llama-2-7b-chat-hf",
#   "messages": [{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": "Say 'test'."}],
#   "temperature": 0.0
# }

# with s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body) as resp:
#     print(resp.json())

In [20]:
%%time
#results = data_sel.apply(lambda x: llm(prompt.format(text=x["text"])), axis=1)

cnt = 0
# cnt = 54

results = []

for index, row in data_sel.iterrows():

  #print("\n\n##################################################")

  #print("EXPECTED_SDG:", row["sdg"], "TEXT:", row["text"].replace('\n',''),'\n\n')

  few_shots_list = top7_train_data_sim_to_each_test_data[row['text_id']][:number_of_shots]

  few_shots = ""

  for train_data_id in few_shots_list:

    # print("##################################################")
    # print(train_data_id)

    train_row = data_sel_4_few_shot_examples[data_sel_4_few_shot_examples.text_id==train_data_id]

    # print(train_row)

    # print(train_row.text.values[0])
    # print(train_row.sdg.values[0])

    input_text = "Input text: \""+train_row.text.values[0].replace('\n','')+"\"\n"
    output_text = "Output: \"---SDG "+str(train_row.sdg.values[0])+'---\"\n\n'

    few_shots += input_text
    few_shots += output_text

  #print(few_shots)

  template = """[INST]
<<SYS>>
Classify the following text between triple quotes according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015, choosing one (and only one) of the SDG labels based on it best representing the content of the entire text enclosed in triple quotes.

In the following lines, a text enclosed in triple quotes will be given to you.

The task is to classify the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015.

The constraint to adhere to is that for the classification, you must select only one out of all the possible SDGs.

If you believe that the text to be processed cannot be classified under any of the possible SDGs, then classify it as "SDG 0".

The following are """+str(number_of_shots)+""" non exhaustive examples of expected classification given an input text:

"""+few_shots+"""


The text to be processed is:
<</SYS>>

'''
"""+row["text"]+"""
'''

<<SYS>>
Answer with label only usign the format:---LABEL---.
<</SYS>>
[/INST]
"""



  # print(template)

  text_id = row['text_id']
  text = row["text"]
  sdg_true = row["sdg"]

  #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
  query = template

  body = {
    "model": model_name,
    "messages": [
        {"role": "system", "content": "You are a helpful assistant."}, 
        {"role": "user", "content": query}],
    "temperature": 0.0
  }


  try:
      result = s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body).json()
      print(cnt, text_id, sdg_true, result['choices'][0]['message']['content'].replace("\n"," - "))
  except Exception as e:
#       print("#############################################################################")
#       print(e, result)
      print(cnt, text_id, sdg_true, "ERROR:"+str(e)+str(result))
#       print("#############################################################################")
  #$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$
    
  result_extended = (cnt, text_id, sdg_true, result)

  #print(result_extended)

  with open(output_datapath+f'/sdg_classification_Llama2_7B_SEMANTIC_SIMILARITY_{cnt}.pkl', 'wb') as f:
      pickle.dump(result_extended, f)

  results.append(result_extended)

  cnt+=1

  # if cnt==5:
  #   break

0 6668 0   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs): -  - ---SDG 11--- -  - The text describes a subway train derailing in Brooklyn, with no injuries reported. The focus is on the restoration of service and the impact on commuters. The SDG that best represents this text is SDG 11, as it deals with sustainable cities and communities, including the need for reliable and safe public transportation systems.
1 6099 0   Sure, I'd be happy to help! Here's the text you provided: -  - Brooklyn Decker’s SI Swimsuit Cover Photo Shoot in the Maldives Is Beyond Beautiful  Yahoo Life -  - Based on the content of the text, I would classify it as: -  - ---SDG 11--- -  - The text does not mention any specific sustainable development goals, but it does describe a beautiful and luxurious vacation in the Maldives, which is a popular tourist destination known for its beaches and crystal-clear waters. The text also mentions

20 1376 0   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015: -  - ---SDG 0--- -  - The given text does not mention any specific SDGs or related issues, and it does not provide any information that could be linked to any of the SDGs. Therefore, the best label for this text is "SDG 0".
21 3889 0   Sure! Here's the answer with the label you requested: -  - ---SDG 0--- -  - The text you provided does not mention any of the United Nations' Sustainable Development Goals (SDGs) adopted in 2015. Therefore, the label is SDG 0, which represents the absence of any SDGs.
22 6576 0   Sure, I'd be happy to help! Here's the text you provided: -  - "A.I. May Someday Work Medical Miracles. For Now, It Helps Do Paperwork. The New York Times" -  - Based on the content of the text, I would classify it as: -  - ---SDG 3--- -  - The text discusses the use of artificial intelligen

41 6772 0 ERROR:'choices'{'generated_text': None, 'embedding_outputs': None, 'num_input_tokens': None, 'num_input_tokens_batch': None, 'num_generated_tokens': None, 'num_generated_tokens_batch': None, 'preprocessing_time': None, 'generation_time': None, 'timestamp': 1701194568.4059308, 'finish_reason': None, 'error': {'message': 'rayllm.backend.llm.error_handling.PromptTooLongError: Input too long. Recieved 5240 tokens, but the maximum input length is 4096 tokens. (Request ID: a923aa4183acf0630272883dc664c222)', 'internal_message': 'rayllm.backend.server.openai_compat.openai_exception.OpenAIHTTPException (Request ID: a923aa4183acf0630272883dc664c222)', 'code': 400, 'type': 'OpenAIHTTPException', 'param': {}}, 'num_total_tokens': 0, 'num_total_tokens_batch': 0, 'total_time': None}
42 6340 0 ERROR:'choices'{'generated_text': None, 'embedding_outputs': None, 'num_input_tokens': None, 'num_input_tokens_batch': None, 'num_generated_tokens': None, 'num_generated_tokens_batch': None, 'preproc

60 6573 1   Sure, I'd be happy to help! Here's the label for the given text: -  - ---SDG 1---
61 357 1   Sure, I'd be happy to help! Here's the text you provided, with my classification of it according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015: -  - "The indigence rate in Panama increased by 1.5 percentage points, while the other countries with information available (the Dominican Republic, El Salvador and Paraguay) did not register significant changes in their poverty and indigence rates (see figure 1.3). Panama is an exception, as despite the absence of significant changes in the poverty rate it registers an appreciable rise in the poverty gap squared index, revealing a worsening of the situation among the poorest. The poverty gap squared index also rose by slightly more than the poverty' rate in Mexico." -  - ---SDG 1---
62 175 1   Sure, I'd be happy to help! Here's the text you provided, with my classification of it according to

74 1250 1   Sure, I'd be happy to help! Here's the text you provided: -  - "Results for countries using national panels added additional controls - industry, firm size and job tenure (for Australia, Germany and the United Kingdom). The wage differentials between other age/skill/contract groups can be obtained in a similar way. Table 4.4 provides information on annual changes of contract and the unadjusted rates of moving up/down or staying in the same earnings quintiles over any two consecutive years." -  - Based on the content of the text, I would classify it as: -  - ---SDG 10--- -  - The text discusses wage differentials and employment rates, which are key indicators for SDG 10 (Reduced Inequalities).
75 2799 1   Sure, I'd be happy to help! Here's the text you provided, with my classification of the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015: -  - ---SDG 1--- -  - The text discusses the first target of the first SDG

...

923 1299 15   Sure, I'd be happy to help! Here's the text you provided, with my classification of it according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015: -  - ---SDG 15--- -  - The text discusses the need for closer and more consistent international guidance, rules, and codes of conduct on legality, due diligence, and forest certification, particularly for small and medium-sized enterprises (SMEs). It also mentions the challenges faced by SMEs in following the latest international developments. The text is focused on the sustainable use and management of forests, which is aligned with SDG 15: Life On Land.
924 5980 15   Sure, I'd be happy to help! Here's the classification of the given text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015: -  - ---SDG 15--- -  - The text discusses the classification of ecosystems, the availability of resources and services, and the lack of inf

936 3947 15   Sure, I'd be happy to help! Here's the text you provided, with my classification of it according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015: -  - ---SDG 15--- -  - The text discusses the impact of agricultural land expansion on forests, specifically in the context of Cambodia. The mention of mining and deforestation also suggests that the text could be classified under SDG 15, as it relates to the conservation and sustainable use of terrestrial ecosystems, forests, and wildlife.
937 3491 15   Sure, I'd be happy to help! Here's the text you provided, with my classification of it according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015: -  - "It follows that the estimated total revenue of TSh 68,173,076,760 translates to TSh 1,417 per hectare per year (2013). Following the methodology developed above, we can infer that TFS spends at least TSh 1,006 per ha per year (2013) to

950 518 15   Sure, I'd be happy to help! Here's the label for the given text: -  - ---SDG 15---
951 2066 15   Sure, I'd be happy to help! Here's the classification of the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015: -  - ---SDG 15--- -  - The text discusses the importance of ecotourism as a sector with green growth potential and the need to explore various instruments to capture the international public good benefits provided by protected areas. This aligns with SDG 15, which focuses on life on land and aims to protect, restore, and promote the sustainable use of terrestrial ecosystems, forests, forests, wetlands, and wildlife.
952 389 15   Sure, I'd be happy to help! Here's the classification of the text according to the Sustainable Development Goals (SDGs) adopted by all United Nations Member States in 2015: -  - ---SDG 15--- -  - The text mentions the importance of biodiversity and the loss of biodiversity in Scotlan

976 3236 16   Sure, I'd be happy to help! Here's the label for the text you provided: -  - ---SDG 16---
977 6715 16   Sure, I'd be happy to help! Based on the text you provided, I would classify it as follows: -  - ---SDG 16--- -  - The text discusses how balance-of-power politics and geopolitical calculations have influenced the responses to the covid-19 pandemic, and how this has impacted international health cooperation and the use of international law in addressing health issues. The focus on geopolitics and power dynamics is consistent with SDG 16, which aims to promote peaceful and inclusive societies, access to justice, and effective and accountable institutions.
978 2543 16 ---SDG 16---
979 64 16   Sure, I'd be happy to help! Based on the text you provided, I would classify it as follows: -  - ---SDG 16--- -  - The text focuses on the examination of the United Nations Security Council's (UNSC) process in the aftermath of the 9/11 attacks, specifically looking at the adoption of

1006 4967 16   Sure, I'd be happy to help! Here's the label for the text you provided: -  - ---SDG 16--- -  - The text describes the story of Strive Masiyiwa's quest to obtain a mobile telecommunications license in Zimbabwe, and how he faced obstacles from the government and the courts. The text highlights the importance of ethical means and the independence of the judiciary in obtaining the license. The story represents a successful example of resistance to political corruption, which aligns with SDG 16 (peace, justice, and strong institutions).
1007 1794 16   Sure, I'd be happy to help! Here's the label for the text you provided: -  - ---SDG 5---
1008 1864 16   Sure, I'd be happy to help! Based on the text you provided, the most appropriate SDG label would be: -  - ---SDG 16--- -  - This label represents the theme of "Peace, Justice and Strong Institutions," which is the focus of SDG 16. The text discusses the development of international criminal law and the rights of individuals in

In [21]:
# import time

# cnt = 0
# # cnt = 181

# results = []
# for index, row in data_sel.iterrows():
    
#     text_id = row['text_id']
#     text = row["text"]
#     sdg_true = row["sdg"]
    
#     query = template_part1 + text + template_part2
    
#     body = {
#       "model": "meta-llama/Llama-2-13b-chat-hf",
#       "messages": [
#           {"role": "system", "content": "You are a helpful assistant."}, 
#           {"role": "user", "content": query}],
#       "temperature": 0.0
#     }
    
        
#     try:
#         result = s.post(url, headers={"Authorization": f"Bearer {token}"}, json=body).json()
#         print(cnt, text_id, sdg_true, result['choices'][0]['message']['content'])
#     except Exception as e:
#         print("#############################################################################")
#         print(e)
#         print("#############################################################################")
        
    
#     result_extended = (cnt, text_id, sdg_true, result)
    
#     with open(output_datapath+f'sdg_classification_Llama2_13B_{cnt}.pkl', 'wb') as f:
#         pickle.dump(result_extended, f)
    
#     results.append(result_extended)
#     cnt+=1
#     time.sleep(0.5)

In [22]:
files = glob.glob(output_datapath + "*.pkl")
len(files)

1020

In [23]:
# df_list = (pd.DataFrame(pd.read_pickle(file), columns=['cnt', 'id', 'true_label', 'predicted_label']) for file in files)
# df_list = (pd.DataFrame(pd.read_pickle(file)) for file in files)

tuple_list = []
for file in files:
    res = pd.read_pickle(file)
    try:
        if len(res[3]['choices'])>1:
            raise Exception('ERROR: Multilabel detected!')
            break
        sdg_pred = res[3]['choices'][0]['message']['content']
        res4df = (res[0], res[1], res[2], sdg_pred)
        print(res4df)
        tuple_list.append(res4df)
    except: 
        print("#################################################################")
        print(res)
        print("#################################################################")
        

(289, 1960, 4, '  Sure, I\'d be happy to help! Here\'s the text you provided, with my classification of it according to the Sustainable Development Goals (SDGs):\n\n"While numbers of teachers have increased in the last ten years, the percentage of qualified teachers has dropped significantly (Education International, 2007). This could be attributed to the rise in enrolments after the introduction of Free Primary Education in 2000. As a result, while access has increased between 1999 - 2008 with an old NER of 57 compared to 73 respectively (GMR, 2010), the issue of quality in education is a prevalent one."\n\nBased on the content of the text, I would classify it as follows:\n\n---SDG 4---\n\nThe text discusses the issue of quality in education, specifically the decrease in the percentage of qualified teachers, and the impact of free primary education on access to education. These issues are all relevant to SDG 4, which aims to ensure inclusive and equitable quality education and promote

In [24]:
len(tuple_list)

1012

In [25]:
results_processed = pd.DataFrame(tuple_list, columns=['cnt', 'id', 'true_label', 'predicted_label'])

In [26]:
results_processed

,cnt,id,true_label,predicted_label
0,289,1960,4,"Sure, I'd be happy to help! Here's the text ..."
1,783,2676,13,"Sure, I'd be happy to help! Here's the label..."
2,246,3956,4,"Sure, I'd be happy to help! Here's the text ..."
3,50,2541,0,"Sure, I'd be happy to help! Here's the class..."
4,288,4205,4,"Sure, I'd be happy to help! Here's the label..."
...,...,...,...,...
1007,691,5312,11,"Sure, I'd be happy to help! Here's the text ..."
1008,137,2747,2,"Sure, I'd be happy to help! Here's the class..."
1009,513,4489,8,"Sure, I'd be happy to help! Here's the class..."
1010,723,2110,12,"Sure, I'd be happy to help! Here's the text ..."


In [27]:
results_processed_merged = data.merge(results_processed, right_on='id', left_on='text_id')
results_processed_merged

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,id,true_label,predicted_label
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,0,6668,0,"Sure, I'd be happy to help! Here's the class..."
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,1,6099,0,"Sure, I'd be happy to help! Here's the text ..."
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,2,6471,0,---SDG 16---
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,3,1786,0,"Sure, I'd be happy to help! Here's the label..."
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN,4,2143,0,"Sure, I'd be happy to help! Here's the label..."
...,...,...,...,...,...,...,...,...,...,...,...
1007,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,1015,4695,16,"Sure, I'd be happy to help! Here's the label..."
1008,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,1016,6628,16,"Sure, I'd be happy to help! Based on the tex..."
1009,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,1017,4563,16,"Sure, I'd be happy to help! Here's the label..."
1010,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,1018,6710,16,"Sure, I'd be happy to help! Here's the label..."


In [28]:
results_processed_merged['predicted_label_clean'] = results_processed_merged['predicted_label'].str.extract(r'SDG (\d+)')
results_processed_merged

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,id,true_label,predicted_label,predicted_label_clean
0,6668,NaN,L train resumes service after derailing from t...,0,NaN,NaN,NaN,0,6668,0,"Sure, I'd be happy to help! Here's the class...",11
1,6099,NaN,Brooklyn Decker’s SI Swimsuit Cover Photo Shoo...,0,NaN,NaN,NaN,1,6099,0,"Sure, I'd be happy to help! Here's the text ...",11
2,6471,NaN,Mumbai: The Bombay High Court has suspended th...,0,NaN,NaN,NaN,2,6471,0,---SDG 16---,16
3,1786,NaN,Four minor planets named after Indian scientis...,0,NaN,NaN,NaN,3,1786,0,"Sure, I'd be happy to help! Here's the label...",13
4,2143,NaN,By Dipo Olowookere\n\nThe President of the Man...,0,NaN,NaN,NaN,4,2143,0,"Sure, I'd be happy to help! Here's the label...",7
...,...,...,...,...,...,...,...,...,...,...,...,...
1007,4695,10.1080/0961452032000170794,Human rights NGOs were the vanguard of the str...,16,0.0,6.0,1.0,1015,4695,16,"Sure, I'd be happy to help! Here's the label...",16
1008,6628,10.1057/9781137023735_15,An important transformation took place in inte...,16,0.0,6.0,1.0,1016,6628,16,"Sure, I'd be happy to help! Based on the tex...",16
1009,4563,10.33751/JHSS.V4I1.1904,Accountability of government officials' action...,16,0.0,6.0,1.0,1017,4563,16,"Sure, I'd be happy to help! Here's the label...",16
1010,6710,10.2139/SSRN.460480,"In Phillips Petroleum Co. v. Shutts, the Unite...",16,0.0,5.0,1.0,1018,6710,16,"Sure, I'd be happy to help! Here's the label...",16


In [29]:
results_processed_merged.groupby("predicted_label_clean").count()

,text_id,doi,text,sdg,labels_negative,labels_positive,agreement,cnt,id,true_label,predicted_label
predicted_label_clean,,,,,,,,,,,
0,18,0,18,18,0,0,0,18,18,18,18
1,77,77,77,77,77,77,77,77,77,77,77
10,47,46,47,47,46,46,46,47,47,47,47
11,70,63,70,70,63,63,63,70,70,70,70
12,66,65,66,66,65,65,65,66,66,66,66
13,60,59,60,60,59,59,59,60,60,60,60
14,61,60,61,61,60,60,60,61,61,61,61
15,64,62,64,64,62,62,62,64,64,64,64
16,64,60,64,64,60,60,60,64,64,64,64


In [30]:
results_processed_merged.to_excel(output_datapath+'report_sdg_classification_Llama2_7B.xlsx')

In [31]:
import numpy as np
from sklearn.metrics import classification_report

y_true = results_processed_merged.sdg 
y_pred = results_processed_merged.predicted_label_clean
y_true = np.array([int(label) for label in y_true])
y_pred = np.array([int(label) for label in y_pred])
print(y_true.shape[0])
print(y_pred.shape[0])
#target_names = ['sdg 0', 'sdg 1', 'sdg 2',  'sdg 3']

print(classification_report(y_true, y_pred)) #, target_names=target_names))

1012
1012
              precision    recall  f1-score   support

           0       1.00      0.35      0.51        52
           1       0.70      0.90      0.79        60
           2       0.91      0.83      0.87        60
           3       0.81      0.93      0.87        60
           4       0.86      0.92      0.89        60
           5       0.90      0.95      0.93        60
           6       1.00      0.95      0.97        60
           7       0.95      0.87      0.90        60
           8       0.68      0.70      0.69        60
           9       0.84      0.82      0.83        60
          10       0.79      0.62      0.69        60
          11       0.79      0.92      0.85        60
          12       0.86      0.95      0.90        60
          13       0.90      0.90      0.90        60
          14       0.93      0.95      0.94        60
          15       0.86      0.92      0.89        60
          16       0.86      0.92      0.89        60
          17     

/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/opt/conda/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
